In [9]:
from crewai import Agent,Task,Crew,Process
from crewai_tools import SerperDevTool
from langchain_groq import ChatGroq
import os
from dotenv import load_dotenv

In [10]:
load_dotenv()
GROQ_API_KEY= os.getenv("GROQ_API_KEY")
SERPER_API_KEY= os.getenv("SERPER_API_KEY")

In [12]:
llm = ChatGroq(model="gemma-7b-it",groq_api_key=os.getenv("GROQ_API_KEY"))
print(llm.invoke("hey! what's up!"))
search_tool = SerperDevTool()

content="Not much, just hanging out in the digital realm, waiting to be of assistance. How about you? What's on your mind today?" response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 21, 'total_tokens': 51, 'completion_time': 0.032843465, 'prompt_time': 0.012722541, 'queue_time': None, 'total_time': 0.045566006000000006}, 'model_name': 'gemma-7b-it', 'system_fingerprint': 'fp_7d8efeb0b1', 'finish_reason': 'stop', 'logprobs': None} id='run-f636dc6a-cbc9-45b5-8ac2-f719858a15b7-0'


In [13]:
# Define the assignment text and submission

assignment_text = """
Assignment: Personalized Article Summaries

Goal:

Develop a prompt that can summarize news articles into statements of a specified length and focus on information specific to your interests.

Summarize a few news articles yourself. Your summaries should be the length, tone, and writing style that you prefer. Make sure to discuss the information that you personally find interesting from the articles. Then, use your example summaries for in-context learning with a language model like OpenAI's ChatGPT, Google's Bard, or Anthropic's Claude. Now, the language model should learn from the context how to create news summaries that fit your length, tone, style, and content preferences.
"""

assignmentsubmission = """
Assignment Submission:
- Article Title: "Africa's Great Green Wall Initiative Making Significant Progress"
  Summary: The Great Green Wall, an ambitious project to combat desertification in Africa, has achieved significant milestones. Over 20 million hectares of land have been restored, benefiting local communities through increased agricultural productivity and job creation. The initiative aims to restore 100 million hectares by 2030, contributing to climate resilience and biodiversity.
- Article Title: "Revolutionary AI Developed for Early Disease Detection"
  Summary: Researchers have developed a groundbreaking AI system capable of early detection of diseases like cancer and Alzheimer's. Using advanced algorithms and vast datasets, the AI can analyze medical images with unprecedented accuracy, potentially saving millions of lives through early intervention. This innovation marks a significant leap forward in personalized medicine and healthcare efficiency.
- Article Title: "Astronomers Discover Earth-like Exoplanets in Habitable Zone"
  Summary: A team of astronomers has discovered two Earth-like exoplanets within the habitable zone of a nearby star. These planets, located 12 light-years away, have conditions that may support liquid water, raising the possibility of life beyond our solar system. This discovery fuels excitement in the search for extraterrestrial life and understanding planetary formation.
"""

# Define agents for each grading criterion
clarity_agent = Agent(
    role='Clarity and Conciseness Grader',
    goal='Evaluate the clarity and conciseness of the assignment submission',
    backstory=f"""You are an experienced educator with a keen eye for detail, specializing in evaluating the clarity and conciseness of written content.
    Assignment: {assignment_text}""",
    verbose=True,
    llm=llm,
    allow_delegation=False
)

relevance_agent = Agent(
    role='Relevance and Focus Grader',
    goal='Evaluate the relevance and focus of the assignment submission',
    backstory=f"""You are a seasoned educator with a strong background in assessing the relevance and focus of academic work.
    Assignment: {assignment_text}""",
    verbose=True,
    llm=llm,
    allow_delegation=False
)

accuracy_agent = Agent(
    role='Accuracy Grader',
    goal='Evaluate the accuracy of the information in the assignment submission',
    backstory=f"""You are an expert in evaluating the accuracy of content, ensuring that all facts and details are correctly represented.
    Assignment: {assignment_text}""",
    verbose=True,
    llm=llm,
    allow_delegation=False
)

tone_agent = Agent(
    role='Tone and Style Grader',
    goal='Evaluate the tone and style of the assignment submission',
    backstory=f"""You have a background in literature and writing, with extensive experience in evaluating the tone and style of written work.
    Assignment: {assignment_text}""",
    verbose=True,
    llm=llm,
    allow_delegation=False
)

examples_agent = Agent(
    role='Examples and Details Grader',
    goal='Evaluate the use of examples and details in the assignment submission',
    backstory=f"""You specialize in assessing the use of examples and details in academic and professional writing.
    Assignment: {assignment_text}""",
    verbose=True,
    llm=llm,
    allow_delegation=False
)

# Create tasks for each agent
task_clarity = Task(
    description=f"""Grade the clarity and conciseness of the following assignment submission on a scale of 1-20:
    {assignmentsubmission}""",
    expected_output="Score for clarity and conciseness with comments",
    agent=clarity_agent
)

task_relevance = Task(
    description=f"""Grade the relevance and focus of the following assignment submission on a scale of 1-30:
    {assignmentsubmission}""",
    expected_output="Score for relevance and focus with comments",
    agent=relevance_agent
)

task_accuracy = Task(
    description=f"""Grade the accuracy of the information in the following assignment submission on a scale of 1-20:
    {assignmentsubmission}""",
    expected_output="Score for accuracy with comments",
    agent=accuracy_agent
)

task_tone = Task(
    description=f"""Grade the tone and style of the following assignment submission on a scale of 1-20:
    {assignmentsubmission}""",
    expected_output="Score for tone and style with comments",
    agent=tone_agent
)

task_examples = Task(
    description=f"""Grade the use of examples and details in the following assignment submission on a scale of 1-10:
    {assignmentsubmission}""",
    expected_output="Score for use of examples and details with comments",
    agent=examples_agent
)

# Instantiate your crew with a sequential process
crew = Crew(
    agents=[clarity_agent, relevance_agent, accuracy_agent, tone_agent, examples_agent],
    tasks=[task_clarity, task_relevance, task_accuracy, task_tone, task_examples],
    verbose=2, # You can set it to 1 or 2 to different logging levels
    process=Process.sequential
)

# Get your crew to work!
result = crew.kickoff()

print("######################")
print(result)


 [2024-07-05 20:24:40][DEBUG]: == Working Agent: Clarity and Conciseness Grader
 [2024-07-05 20:24:40][INFO]: == Starting Task: Grade the clarity and conciseness of the following assignment submission on a scale of 1-20:
    
Assignment Submission:
- Article Title: "Africa's Great Green Wall Initiative Making Significant Progress"
  Summary: The Great Green Wall, an ambitious project to combat desertification in Africa, has achieved significant milestones. Over 20 million hectares of land have been restored, benefiting local communities through increased agricultural productivity and job creation. The initiative aims to restore 100 million hectares by 2030, contributing to climate resilience and biodiversity.
- Article Title: "Revolutionary AI Developed for Early Disease Detection"
  Summary: Researchers have developed a groundbreaking AI system capable of early detection of diseases like cancer and Alzheimer's. Using advanced algorithms and vast datasets, the AI can analyze medical im

In [14]:
researcher = Agent(
    role = "Researcher",
    goal='Pioneer revolutionary advancements in {subject_area}',
    backstory=(
    "As a visionary researcher, your insatiable curiosity drives you"
    "to delve deep into emerging fields. With a passion for innovation"
    "and a dedication to scientific discovery, you seek to"
    "develop technologies and solutions that could transform the future."),
    llm = llm,
    max_iter = 5,
    tools = [search_tool],
    allow_delegation=True,
    verbose=True
)

writer = Agent(
    role = "Writer",
    goal='Craft engaging and insightful narratives about {subject_area}',
    verbose=True,
    backstory=(
    "You are a skilled storyteller with a talent for demystifying"
    "complex innovations. Your writing illuminates the significance"
    "of new technological discoveries, connecting them with everyday"
    "lives and broader societal impacts."
    ),
    tools = [search_tool],
    llm = llm,
    max_iter = 5,
    allow_delegation=False
)

In [15]:
research_task = Task(
    description = (
    "Explore and identify the major development within {subject_area}"
    "Detailed SEO report of the development in a comprehensive narrative."
  ),
    expected_output='A report, structured into three detailed paragraphs',
    tools = [search_tool],
    agent = researcher
)

write_task = Task(
    description=(
    "Craft an engaging article on recent advancements  within {subject_area}"
    "The article should be clear, captivating, and optimistic, tailored for a broad audience."
  ),
    expected_output='A blog article on recent advancements in {subject_area} in markdown.',
    tools = [search_tool],
    agent = writer,
    async_execution = False,
    output_file = "blog.md"
)

In [ ]:
crew = Crew(
    agents = [researcher,writer],
    tasks = [research_task,write_task],
    process = Process.sequential,
    max_rpm = 3,
    cache=True
)

result = crew.kickoff(inputs={'subject_area':"Genrative AI"})
print(result)